# Object-Augmented Schemas

This tutorial covers DataJoint's Object-Augmented Schema (OAS) model. You'll learn:

- **The OAS concept** — Unified relational + object storage
- **Blobs** — Storing arrays and Python objects
- **Object storage** — Scaling to large datasets
- **Staged insert** — Writing directly to object storage (Zarr, HDF5)
- **Attachments** — Preserving file names and formats
- **Codecs** — How data is serialized and deserialized

In an Object-Augmented Schema, the relational database and object storage operate as a **single integrated system**—not as separate "internal" and "external" components.

In [ ]:
import datajoint as dj
import numpy as np

schema = dj.Schema('tutorial_oas')

## The Object-Augmented Schema Model

Scientific data often combines:
- **Structured metadata** — Subjects, sessions, parameters (relational)
- **Large data objects** — Arrays, images, recordings (binary)

DataJoint's OAS model manages both as a unified system:

```mermaid
block-beta
    columns 1
    block:oas:1
        columns 2
        OAS["Object-Augmented Schema"]:2
        block:db:1
            DB["Relational Database"]
            DB1["Metadata"]
            DB2["Keys"]
            DB3["Relationships"]
        end
        block:os:1
            OS["Object Storage (S3/File/etc)"]
            OS1["Large arrays"]
            OS2["Images/videos"]
            OS3["Recordings"]
        end
    end
```

From the user's perspective, this is **one schema**—storage location is transparent.

## Blob Attributes

Use `<blob>` to store arbitrary Python objects:

In [ ]:
@schema
class Recording(dj.Manual):
    definition = """
    recording_id : int
    ---
    metadata : <blob>         # Dict, stored in database
    waveform : <blob>         # NumPy array, stored in database
    """

In [ ]:
# Insert with blob data
Recording.insert1({
    'recording_id': 1,
    'metadata': {'channels': 32, 'sample_rate': 30000, 'duration': 60.0},
    'waveform': np.random.randn(32, 30000)  # 32 channels x 1 second
})

Recording()

In [ ]:
# Fetch blob data
data = (Recording & {'recording_id': 1}).fetch1()
print(f"Metadata: {data['metadata']}")
print(f"Waveform shape: {data['waveform'].shape}")

### What Can Be Stored in Blobs?

The `<blob>` codec handles:

- NumPy arrays (any dtype, any shape)
- Python dicts, lists, tuples, sets
- Strings, bytes, integers, floats
- datetime objects and UUIDs
- Nested combinations of the above

**Note:** Pandas DataFrames should be converted before storage (e.g., `df.to_dict()` or `df.to_records()`).

In [ ]:
@schema
class AnalysisResult(dj.Manual):
    definition = """
    result_id : int
    ---
    arrays : <blob>
    nested_data : <blob>
    """

# Store complex data structures
arrays = {'x': np.array([1, 2, 3]), 'y': np.array([4, 5, 6])}
nested = {'arrays': [np.array([1, 2]), np.array([3, 4])], 'params': {'a': 1, 'b': 2}}

AnalysisResult.insert1({
    'result_id': 1,
    'arrays': arrays,
    'nested_data': nested
})

# Fetch back
result = (AnalysisResult & {'result_id': 1}).fetch1()
print(f"Arrays type: {type(result['arrays'])}")
print(f"Arrays keys: {result['arrays'].keys()}")

## Object Storage with `@`

For large datasets, add `@` to route data to object storage. The schema remains unified—only the physical storage location changes.

### Configure Object Storage

First, configure a store:

In [ ]:
import tempfile
import os

# Create a store for this tutorial
store_path = tempfile.mkdtemp(prefix='dj_store_')

# Configure object storage (global settings for staged_insert1)
dj.config.object_storage.protocol = 'file'
dj.config.object_storage.location = store_path
dj.config.object_storage.project_name = 'tutorial'

# Also configure as a named store for <blob@tutorial> syntax
dj.config.object_storage.stores['tutorial'] = {
    'protocol': 'file',
    'location': store_path
}

print(f"Store configured at: {store_path}")

### Using Object Storage

In [ ]:
@schema
class LargeRecording(dj.Manual):
    definition = """
    recording_id : int
    ---
    small_data : <blob>            # In database (small)
    large_data : <blob@tutorial>   # In object storage (large)
    """

In [ ]:
# Insert data - usage is identical regardless of storage
small = np.random.randn(10, 10)
large = np.random.randn(1000, 1000)  # ~8 MB array

LargeRecording.insert1({
    'recording_id': 1,
    'small_data': small,
    'large_data': large
})

LargeRecording()

In [ ]:
# Fetch is also identical - storage is transparent
data = (LargeRecording & {'recording_id': 1}).fetch1()
print(f"Small data shape: {data['small_data'].shape}")
print(f"Large data shape: {data['large_data'].shape}")

In [ ]:
# Objects are stored in the configured location
for root, dirs, files in os.walk(store_path):
    for f in files:
        path = os.path.join(root, f)
        size = os.path.getsize(path)
        print(f"{os.path.relpath(path, store_path)}: {size:,} bytes")

### Content-Addressed Storage

`<blob@>` uses content-addressed (hash-based) storage. Identical data is stored only once:

In [ ]:
# Insert the same data twice
shared_data = np.ones((500, 500))

LargeRecording.insert([
    {'recording_id': 2, 'small_data': small, 'large_data': shared_data},
    {'recording_id': 3, 'small_data': small, 'large_data': shared_data},  # Same!
])

print(f"Rows in table: {len(LargeRecording())}")

# Deduplication: identical data stored once
files = [f for _, _, fs in os.walk(store_path) for f in fs]
print(f"Files in store: {len(files)}")

## Path-Addressed Storage with `<object@>`

While `<blob@>` uses content-addressed (hash-based) storage with deduplication, `<object@>` uses **path-addressed** storage where each row has its own dedicated storage path:

| Aspect | `<blob@>` | `<object@>` |
|--------|-----------|-------------|
| Addressing | By content hash | By primary key |
| Deduplication | Yes | No |
| Deletion | Garbage collected | With row |
| Use case | Arrays, serialized objects | Zarr, HDF5, multi-file outputs |

Use `<object@>` when you need:
- Hierarchical formats like Zarr or HDF5
- Direct write access during data generation
- Each row to have its own isolated storage location

In [ ]:
@schema
class ImagingSession(dj.Manual):
    definition = """
    subject_id : int32
    session_id : int32
    ---
    n_frames : int32
    frame_rate : float32
    frames : <object@tutorial>    # Zarr array stored at path derived from PK
    """

### Staged Insert for Direct Object Storage Writes

For large datasets like multi-GB imaging recordings, copying data from local storage to object storage is inefficient. The `staged_insert1` context manager lets you **write directly to object storage** before finalizing the database insert:

1. Set primary key values in `staged.rec`
2. Get a storage handle with `staged.store(field, extension)`
3. Write data directly (e.g., with Zarr)
4. On successful exit, metadata is computed and the record is inserted

In [ ]:
import zarr

# Simulate acquiring imaging data frame-by-frame
n_frames = 100
height, width = 512, 512

with ImagingSession.staged_insert1 as staged:
    # Set primary key values first
    staged.rec['subject_id'] = 1
    staged.rec['session_id'] = 1
    
    # Get storage handle for the object field
    store = staged.store('frames', '.zarr')
    
    # Create Zarr array directly in object storage
    z = zarr.open(store, mode='w', shape=(n_frames, height, width), 
                  chunks=(10, height, width), dtype='uint16')
    
    # Write frames as they are "acquired"
    for i in range(n_frames):
        z[i] = np.random.randint(0, 4096, (height, width), dtype='uint16')
    
    # Set remaining attributes
    staged.rec['n_frames'] = n_frames
    staged.rec['frame_rate'] = 30.0

# Record is now inserted with metadata computed from the Zarr
ImagingSession()

In [ ]:
# Fetch returns an ObjectRef for lazy access
ref = (ImagingSession & {'subject_id': 1, 'session_id': 1}).fetch1('frames')
print(f"Type: {type(ref).__name__}")
print(f"Path: {ref.path}")

# Open as Zarr array (data stays in object storage)
z = zarr.open(ref.fsmap, mode='r')
print(f"Shape: {z.shape}")
print(f"Chunks: {z.chunks}")
print(f"First frame mean: {z[0].mean():.1f}")

### Benefits of Staged Insert

- **No intermediate copies** — Data flows directly to object storage
- **Streaming writes** — Write frame-by-frame as data is acquired
- **Atomic transactions** — If an error occurs, storage is cleaned up automatically
- **Automatic metadata** — File sizes and manifests are computed on finalize

Use `staged_insert1` when:
- Data is too large to hold in memory
- You're generating data incrementally (e.g., during acquisition)
- You need direct control over storage format (Zarr chunks, HDF5 datasets)

## Attachments

Use `<attach>` to store files with their original names preserved:

In [ ]:
@schema
class Document(dj.Manual):
    definition = """
    doc_id : int
    ---
    report : <attach@tutorial>
    """

In [ ]:
# Create a sample file
sample_file = os.path.join(tempfile.gettempdir(), 'analysis_report.txt')
with open(sample_file, 'w') as f:
    f.write('Analysis Results\n')
    f.write('================\n')
    f.write('Accuracy: 95.2%\n')

# Insert using file path directly
Document.insert1({
    'doc_id': 1,
    'report': sample_file  # Just pass the path
})

Document()

In [ ]:
# Fetch returns path to extracted file
doc_path = (Document & {'doc_id': 1}).fetch1('report')
print(f"Type: {type(doc_path)}")
print(f"Path: {doc_path}")

# Read the content
with open(doc_path, 'r') as f:
    print(f"Content:\n{f.read()}")

## Codec Summary

| Codec | Syntax | Description |
|-------|--------|-------------|
| `<blob>` | In database | Python objects, arrays |
| `<blob@>` | Default store | Large objects, hash-addressed |
| `<blob@name>` | Named store | Specific storage tier |
| `<attach>` | In database | Files with names |
| `<attach@name>` | Named store | Large files with names |
| `<object@name>` | Named store | Path-addressed (Zarr, etc.) |
| `<filepath@name>` | Named store | References to existing files |

## Computed Tables with Large Data

Computed tables commonly produce large results:

In [ ]:
@schema
class ProcessedRecording(dj.Computed):
    definition = """
    -> LargeRecording
    ---
    filtered : <blob@tutorial>     # Result in object storage
    mean_value : float64
    """

    def make(self, key):
        # Fetch source data
        data = (LargeRecording & key).fetch1('large_data')
        
        # Process
        from scipy.ndimage import gaussian_filter
        filtered = gaussian_filter(data, sigma=2)
        
        self.insert1({
            **key,
            'filtered': filtered,
            'mean_value': float(np.mean(filtered))
        })

In [ ]:
ProcessedRecording.populate(display_progress=True)
ProcessedRecording()

## Efficient Data Access

### Fetch Only What You Need

In [ ]:
# Fetch only scalar metadata (fast)
meta = (ProcessedRecording & {'recording_id': 1}).fetch1('mean_value')
print(f"Mean value: {meta}")

In [ ]:
# Fetch large data only when needed
filtered = (ProcessedRecording & {'recording_id': 1}).fetch1('filtered')
print(f"Filtered shape: {filtered.shape}")

### Project Away Large Columns Before Joins

In [ ]:
# Efficient: project to scalar columns before join
result = LargeRecording.proj('recording_id') * ProcessedRecording.proj('mean_value')
result

## Best Practices

### 1. Choose Storage Based on Size

```python
# Small objects (< 1 MB): no @
parameters : <blob>

# Large objects (> 1 MB): use @
raw_data : <blob@>
```

### 2. Use Named Stores for Different Tiers

```python
# Fast local storage for active data
working_data : <blob@fast>

# Cold storage for archives
archived_data : <blob@archive>
```

### 3. Separate Queryable Metadata from Large Data

```python
@schema
class Experiment(dj.Manual):
    definition = """
    exp_id : int
    ---
    # Queryable metadata
    date : date
    duration : float
    n_trials : int
    # Large data
    raw_data : <blob@>
    """
```

### 4. Use Attachments for Files

```python
# Preserves filename
video : <attach@>
config_file : <attach@>
```

## Quick Reference

| Pattern | Use Case |
|---------|----------|
| `<blob>` | Small Python objects |
| `<blob@>` | Large arrays with deduplication |
| `<blob@store>` | Large arrays in specific store |
| `<attach@store>` | Files preserving names |
| `<object@store>` | Path-addressed data (Zarr, HDF5) |

## Next Steps

- [Configure Object Storage](../how-to/configure-storage.md) — Set up S3, MinIO, or filesystem stores
- [Custom Codecs](advanced/custom-codecs.ipynb) — Define domain-specific types
- [Manage Large Data](../how-to/manage-large-data.md) — Performance optimization

In [ ]:
# Cleanup
schema.drop(prompt=False)
import shutil
shutil.rmtree(store_path, ignore_errors=True)